# Churn Prediction with Machine Learning
## Thiago Pádua de Carvalho - 2020007066

### Introdução
Churn é o termo que define a taxa de evasão de clientes de um serviço ou produto fornecido por uma empresa. Ele é um elemento fundamental para medir a fidelidade e a satisfação dos clientes, um indicador importante para a saúde financeira de uma companhia, análise de situação no mercado e consequente tomada de decisões estratégicas.

Entender o comportamento dos clientes e prever sua evasão é um desafio para as empresas, pois esse fenômeno está diretamente relacionado com a perda de receita e viabilidade do negócio. Sendo assim é fundamental ter informações sobre o que leva um cliente a cancelar um serviço para identificar pontos de falha, tal como investir assertivamente em ações de retenção.

Com isso em mente, o objetivo deste trabalho é construir um modelo de Machine Learning para prever o churn, tal como identificar aquele atributos que mais contribuem para a evasão de clientes, permitindo uma visão ampla e detalhada sobre o problema para a posterior tomada de decisões.

### Importação das bibliotecas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

## Conhecendo os dados
O arquivo de treinamento para o conjunto de dados de churn contém uma coleção de 440.882 registros de clientes, juntamente com suas respectivas características e rótulos binários. Cada linha no arquivo de treinamento representa um cliente e inclui as seguintes características: idade, gênero, tempo de serviço, frequência de uso, chamadas de suporte, atraso no pagamento, tipo de assinatura, duração do contrato, gasto total e última interação.

O arquivo de teste para o conjunto de dados de churn consiste em 64.374 registros de clientes e serve como um conjunto de dados separado para avaliar o desempenho e a capacidade de generalização dos modelos de previsão de rotatividade treinados. Cada linha no arquivo de teste corresponde a um cliente e contém o mesmo conjunto de características que o arquivo de treinamento. No entanto, os rótulos de churn não estão incluídos no arquivo de teste, pois são usados para avaliar a precisão e a eficácia dos modelos. O arquivo de teste permite avaliar o poder preditivo dos modelos treinados em dados não vistos e obter insights sobre o quão bem eles se generalizam para novos clientes.

Fonte dos dados: https://www.kaggle.com/datasets/muhammadshahidazeem/customer-churn-dataset?resource=download&select=customer_churn_dataset-testing-master.csv

### Carregamento dos dados

In [5]:
trainingData = pd.read_csv('data/customer_churn_dataset-testing-master.csv')
X_train, y_train = trainingData.drop('Churn', axis=1), trainingData['Churn']

In [6]:
testData = pd.read_csv('data/customer_churn_dataset-testing-master.csv')
X_test, y_test = testData.drop('Churn', axis=1), testData['Churn']
X_test.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20
2,3,47,Male,27,10,2,29,Premium,Annual,757,21
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18
4,5,53,Female,58,24,9,2,Standard,Annual,533,18
